In [1]:
import syft as sy
# third party
import numpy as np
from sys import getsizeof
private_high = np.random.rand(45_000_000 * 4)

In [2]:
our_bytes = private_high.tobytes()

In [3]:
private_high.nbytes / (10**9)

1.44

In [4]:
a = private_high.tobytes()

In [6]:
d = sy.serialize(private_high, to_bytes=True)

fqn='numpy.ndarray'
<function numpy_serialize at 0x10fc328e0> 1440000112


In [63]:
action_object = sy.ActionObject.from_obj(private_high)

In [64]:
msg = sy.serialize(action_object, to_proto=True)

fqn='syft.service.action.numpy.NumpyArrayObject'
attr_name='id' 40
fqn='syft.types.uid.UID'
attr_name='value' 64
fqn='builtins.bytes'
<function <lambda> at 0x177d8afc0> 49
attr_name='syft_action_data_cache' 1440000112
fqn='numpy.ndarray'
<function numpy_serialize at 0x130f16d40> 1440000112
fqn='builtins.tuple'
<function serialize_iterable at 0x130f5b920> 64
fqn='builtins.bytes'
<function <lambda> at 0x177d8afc0> 1351676677
fqn='builtins.int'
<function <lambda> at 0x177d8b2e0> 32
fqn='builtins.str'
<function <lambda> at 0x177d8ade0> 48
attr_name='syft_action_data_node_id' 16
fqn='builtins.NoneType'
<function <lambda> at 0x177d88900> 16
attr_name='syft_action_data_repr_' 132
fqn='builtins.str'
<function <lambda> at 0x177d8ade0> 132
attr_name='syft_action_data_str_' 112
fqn='builtins.str'
<function <lambda> at 0x177d8ade0> 112
attr_name='syft_action_data_type' 432
fqn='builtins.type'
<function serialize_type at 0x177d8b600> 432
attr_name='syft_blob_storage_entry_id' 16
fqn='builtins.NoneT

* 4gb objects
* 6.5gb system
* ~27gb peak serialization  
* ~17gb for serialization

* system ~ 5gb
* object - 5gb
* bytes - 5gb

In [65]:
import syft as sy
from syft.store.blob_storage.seaweedfs import SeaweedFSClientConfig, SeaweedFSClient
from syft.types.blob_storage import CreateBlobStorageEntry
import np

config = SeaweedFSClientConfig(
        host="localhost",  # "http://localhost:8888"
        port=8333,
        access_key="admin",
        secret_key="admin",
        region="us-east-1",
        default_bucket_name="mynode",
        mount_port=4001,
    )
seaweed_client = SeaweedFSClient(config=config)
connection = seaweed_client.connect()

# from 0.1B to 0.5B of elements, it starts to be slow
# serialized data is from 0.8GB to 4GB
mock_high = np.random.random(200_000_000)
print(mock_high.nbytes / (10**9))
ao = sy.ActionObject.from_obj(mock_high)
data = ao.syft_action_data
storage_entry = CreateBlobStorageEntry.from_obj(data)
res = connection.allocate(obj=storage_entry)
print(res)

ModuleNotFoundError: No module named 'np'